# Information
1. This files create a dense vectors (embeddings) from a group of documents
2. Stores them in a vector database
3. Executes simple test to see if able to be used in conjuction with a LLM

Defaults:
1. Filetype: .pdf
2. Models: text-embedding-3-small & gpt-4-mini

# Config

In [3]:
#config
import os
# FILE_PATH_TO_EMBED = os.path.join("pension-martijn-files", "Kader Datakwaliteit - wet toekomst pensioenen.pdf.pdf")
DIRECTORY_TO_EMBED = os.path.join("pension-martijn-files")

COLLECTION_NAME = "DATA_QUALITY_PENSION" #name of the collection
PERSIST_DIRECTORY = os.path.join("vector_stores", "pension-martijn-embeddings")

#os.path.join( "solvency-II-files", "solvency II - level 1 - v2.pdf")

# Embedding Model

In [6]:
from langchain_openai import AzureOpenAIEmbeddings

# Set your Azure OpenAI credentials
embedding_api_key = os.getenv("AZURE_OPENAI_API_KEY")
# print(api_key)

# Create an AzureOpenAIEmbeddings object
embedding_model = AzureOpenAIEmbeddings(
    model="text-embedding-3-small",
    azure_endpoint="https://openai-playground-bjorn.openai.azure.com/",
    api_version="2023-05-15",
    api_key=embedding_api_key
)


# test out the embedding:

# # Create embeddings for some text
# text = "I love learning about computers!"
# text_embeddings = embedding_model.embed_query(text)

# # Print the embeddings
# print(text_embeddings)
# texts = ["Hello, world!", "How are you?"]
# document_embeddings = embeddings.embed_documents(texts)
# print(document_embeddings)

In [7]:
import warnings

# load pdf
# from langchain_community.document_loaders import PyPDFLoader
# loader = PyPDFLoader(FILE_PATH_TO_EMBED)

# load directory of PDFs
from langchain_community.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader(DIRECTORY_TO_EMBED)

pages = []
async for page in loader.alazy_load():
    pages.append(page)


print(f"LOADED DOCUMENT WITH {len(pages)} PAGES")
if len(pages) > 50:
    warnings.warn("DOCUMENT PAGES LOADED IN IS LARGE THAN 100, MAY INCUR SIGNIFICANT COSTS")


for page in pages:
    if page.page_content == None or page.page_content == "":
        warnings.warn("FOUND PAGES IN DOCUMENT WITHOUT PAGE_CONTENT")

LOADED DOCUMENT WITH 130 PAGES


C:\Users\bvbraak\AppData\Local\Temp\ipykernel_2388\1870643632.py:18: UserWarning: DOCUMENT PAGES LOADED IN IS LARGE THAN 100, MAY INCUR SIGNIFICANT COSTS
  warnings.warn("DOCUMENT PAGES LOADED IN IS LARGE THAN 100, MAY INCUR SIGNIFICANT COSTS")


In [8]:
from IPython.display import Markdown
Markdown(pages[1].page_content)

2 
 
 
 
Inhoudsopgave  
1. INLEIDING KADER DATAKWALITEIT  4 
2. OVERZICHT KADER DATAKWALITEIT  8 
3. FASE 1: OPZET DATAKWALITEIT  15 
3.1 Inleiding  15 
3.2 Stappenplan  15 
3.2.1 Datakw aliteitsbeleid  15 
3.2.2 Kritieke data -elementen (KDE)  15 
4. FASE 2: RISICO -INVENTARISATIE EN -BEOORDELING  16 
4.1 Inleiding  16 
4.2 Stappenpla n 16 
4.2.1 Stap 2.1.1: Profiel pensioenuitvoerder  16 
4.2.2 Stap 2.1.2: Profiel deelnemers  19 
4.2.3 Stap 2.2 Risicobeoordeling  20 
4.2.4 Stap 2.3 Vaststellen aanvullende activiteiten  22 
5. FASE 3: DATA -ANALYSES EN DEELWAARNEMINGEN  25 
5.1 Inleiding  25 
5.2 Stappenplan  25 
5.2.1 Stap 3.1: Data profiling  25 
5.2.2 Stap 3.2.a: Uitvoeren generieke analyses  27 
5.2.3 Stap 3.2.b: Bepalen en uitvoeren specifieke analyses  28 
5.2.4 Stap 3.3: Deelwaarnemingen  30 
6. FASE 4: RAPPORTAGE EN BEOORDELING  33 
6.1 Inleiding  33 
6.2 Stappenplan  33 
6.2.1 Stap 4.1: Rapportage en beoordeling  33 
6.2.2 Stap 4.2: Besluitvorming  34 
6.2.3 Stap 4.3: Correcties uitvoeren  35 
6.2.4 Stap 4.4: Voorlopig oordeel bestuur over de datakwaliteit  36

In [9]:
pages[10].__dict__

{'id': None,
 'metadata': {'producer': 'Microsoft® Word voor Microsoft 365',
  'creator': 'Microsoft® Word voor Microsoft 365',
  'creationdate': '2022-10-23T18:44:56+02:00',
  'author': 'Otto Hulst',
  'moddate': '2022-10-23T18:44:56+02:00',
  'source': 'pension-martijn-files\\Kader Datakwaliteit - wet toekomst pensioenen.pdf.pdf',
  'total_pages': 61,
  'page': 10,
  'page_label': '11'},
 'page_content': '11 \n \n \n \nIn onderstaand schema wordt iedere stap per fase kort toegelicht om een beeld te \ngeven van de reeks van werkzaamheden ten behoeve van het aantoonbaar toetsen van \ndatakwaliteit vóór het invaren. Vanaf hoofdstuk 3 zullen de stappen per fase één voor \néén uitgebreider worden toegelicht door in te gaan op de benodigdheden voor de stap \nen de uit te voeren activiteiten. Hierbij wordt door de pensioenuitvoerder elke stap \ndoorlopen en de uitvoering en uitkomsten daarvan worden gedocumenteerd. Op basis \nvan het principe ‘comply or explain’ kan de pensioenuitvoerder on

In [10]:
#for brevity we will drop a few pages from solvency II to test embedding cost for cheap model
from langchain_text_splitters import RecursiveCharacterTextSplitter

print(f"DOCUMENT WITH {len(pages)} PAGES")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,   # Each chunk will be 1000 characters
    chunk_overlap=100  # Overlap of 100 characters between chunks
)

split_pages = text_splitter.transform_documents(pages)

print(f"NUMBER OF CHUNKS: {len(split_pages)}")

DOCUMENT WITH 130 PAGES
NUMBER OF CHUNKS: 372


In [11]:
from langchain_chroma import Chroma
vectorstore = Chroma(COLLECTION_NAME, embedding_model, persist_directory=PERSIST_DIRECTORY)
print(vectorstore._collection.count())
# test_vectorstore = Chroma.from_documents(split_pages[:3], embedding_model, collection_name=COLLECTION_NAME, persist_directory=PERSIST_DIRECTORY)
# print(test_vectorstore._collection.count())

0


In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_chroma import Chroma

# In memory vector store for testing only
# vectorstore = InMemoryVectorStore(embedding_model)
# create a database with vector embedding and documents.

# note: behavior of from_documents is to add if vectorstore already exists
delete_vector_store_check = input("Do you want to delete the vector_store before adding these documents?")
if delete_vector_store_check == "Y":
    Chroma.delete_collection(Chroma(COLLECTION_NAME, embedding_model, persist_directory=PERSIST_DIRECTORY))

# Save a new vector store based on config at top of this file (create_vector_store.ipynb)
check = input("Are you sure you want to run this command? Creating a vectorstore will append to the existing vectorstore and may incur high costs (Y/n)")

if check == "Y":
    print("CREATING A NEW VECTOR STORE AT SPECIFIED LOCATION")
    vectorstore = Chroma.from_documents(split_pages, embedding_model, collection_name=COLLECTION_NAME, persist_directory=PERSIST_DIRECTORY)
    print(vectorstore._collection.count())

CREATING A NEW VECTOR STORE AT SPECIFIED LOCATION
372


# LLM
Test Cases for embedding

In [17]:
import os
from langchain_openai import AzureChatOpenAI

# setup the llm
llm_api_key = os.getenv("GPT_4O_MINI_API_KEY")
# print(llm_api_key)
llm = AzureChatOpenAI(
    model="gpt-4o-mini",
    azure_endpoint="https://openai-playground-bjorn.openai.azure.com/",
    api_version="2025-01-01-preview",
    api_key=llm_api_key
)

# load the vectorstore
from langchain_chroma import Chroma
vectorstore = Chroma(COLLECTION_NAME, embedding_model, persist_directory=PERSIST_DIRECTORY)
print(vectorstore._collection.count())

654


In [18]:
query = "kan je mij een introductie geven tot fase 4: rapportage en beoordeling"
matched_documents = vectorstore.similarity_search(query=query,k=1)

print(f"First documents of all matched documents: {matched_documents[0].__dict__}")

prompt = ""
document_source = []
for idx, document in enumerate(matched_documents):
    document_source.append(document.metadata["source"])
    prompt += f"""
    source {idx}:
    {document.page_content}

    """

print(document_source)
# print(prompt)

prompt += f"""
Based the above mentioned sources, can you provide an answer on the following question?
Question: {query}
Answer: 
"""

print(prompt)

First documents of all matched documents: {'id': '684ca3a6-d6bd-41c5-ba08-89d8b23ea302', 'metadata': {'author': 'Otto Hulst', 'creationdate': '2022-10-23T18:44:56+02:00', 'creator': 'Microsoft® Word voor Microsoft 365', 'moddate': '2022-10-23T18:44:56+02:00', 'page': 32, 'page_label': '33', 'producer': 'Microsoft® Word voor Microsoft 365', 'source': 'pension-martijn-files\\Kader Datakwaliteit - wet toekomst pensioenen.pdf.pdf', 'total_pages': 61}, 'page_content': '33 \n \n \n \n6. Fase 4: Rapportage en beoordeling \n \n6.1 Inleiding \nOp basis van de resultaten uit de vorige fasen wordt een rapportage opgesteld door \nde pensioenuitvoerder conform de daarvoor binnen de pensioenuitvoerder \nvastgestelde governance, waarbij sleutelhouders expliciet worden betrokken. De \nrapportage wordt vervol gens beoordeeld door het bestuur. Het kader onderscheidt \nvoor deze fase de volgende vier stappen:  \n \n1. Rapportage en evaluatie van fase 2 en 3.  \n2. Beoordeling en besluitvorming door het b

In [19]:
response = llm.invoke(prompt)

In [20]:
from IPython.display import Markdown
Markdown(response.content)

Fase 4 van het proces betreft de rapportage en beoordeling, waarbij de focus ligt op het organiseren van de bevindingen uit de eerdere fasen van het project. In deze fase stelt de pensioenuitvoerder een rapport op dat is gebaseerd op de resultaten uit fase 2 en fase 3. Dit rapport wordt opgesteld volgens de vastgestelde governance-structuur binnen de pensioenuitvoerder, waarbij betrokkenheid van sleutelhouders is gegarandeerd. 

De fase omvat vier belangrijke stappen: 

1. De rapportage en evaluatie van de resultaten uit fase 2 en 3.
2. De beoordeling en besluitvorming door het bestuur op basis van de gepresenteerde rapportage.
3. Het onderzoeken, corrigeren en documenteren van benodigde aanpassingen die in eerdere fasen zijn vastgesteld, met een focus op de oorzaak van de problemen (incidenteel of structureel).
4. Een voorlopig oordeel van het bestuur over de kwaliteit van de aanwezige data.

Deze fase is cruciaal voor het waarborgen van de integriteit en betrouwbaarheid van de gegevens en de aanpassing van de processen op basis van gedetailleerde analyses en bestuurlijke beoordeling.